In [ ]:
# System & Python dependencies
!apt -y update -qq
!apt -y install -qq libgl1-mesa-glx wget git

# Clone ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI

# Install Python packages
!pip install -r requirements.txt

In [ ]:
import torch
print("✅ CUDA available:", torch.cuda.is_available())
print("💡 GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

# 📂 Setup local content folder structure
import os

# Define your base folder inside /content
COMFYUI_BASE = '/ComfyUI_Data'

# 📁 Define required subfolders
REQUIRED_FOLDERS = [
    'models/checkpoints',
    'models/controlnet',
    'models/vae',
    'models/upscale_models',
    'models/clip',
    'models/clip_vision',
    'models/configs',
    'models/diffusers',
    'models/diffusion_models',
    'models/embeddings',
    'models/gligen',
    'models/hypernetworks',
    'models/insightface',
    'models/LLM',
    'models/luts',
    'models/onnx',
    'models/photomaker',
    'models/puild',
    'models/sam2',
    'models/sams',
    'models/text_encoders',
    'models/upscale_models',
    'models/vae_approx',
    'models/unet',
    'models/unet_gguf',
    'models/loras',
    'models/style_models',
    'models/ipadapter-flux',
    'custom_nodes',
    'input',
    'output',
    'temp',
    'user'
]

# ✅ Create folders if not exist
for folder in REQUIRED_FOLDERS:
    full_path = os.path.join(COMFYUI_BASE, folder)
    os.makedirs(full_path, exist_ok=True)

print("✅ All required folders are created in /content/ComfyUI_Data")

In [ ]:
# 🔗 Setup symlinks to use local content folder
COMFYUI_PATH = '/ComfyUI'

LINKS = {
    'models': f'{COMFYUI_BASE}/models',
    'custom_nodes': f'{COMFYUI_BASE}/custom_nodes',
    'input': f'{COMFYUI_BASE}/input',
    'output': f'{COMFYUI_BASE}/output',
    'temp': f'{COMFYUI_BASE}/temp',
    'user': f'{COMFYUI_BASE}/user'
}

for name, target in LINKS.items():
    source = os.path.join(COMFYUI_PATH, name)
    # Remove old local folder
    if os.path.islink(source) or os.path.isdir(source):
        !rm -rf "{source}"
    # Create symlink
    os.symlink(target, source)

print("✅ ComfyUI is now configured to use local /content folder.")

In [ ]:
# ComfyUI Manager
!pip install "numpy==1.26.4" --force-reinstall --quiet > /dev/null 2>&1
!rm -rf /ComfyUI/custom_nodes/ComfyUI-Manager
!git clone --quiet https://github.com/ltdrdata/ComfyUI-Manager.git /ComfyUI/custom_nodes/ComfyUI-Manager
print("✅ ComfyUI-Manager installed and numpy version is set for compatibility.")


In [ ]:
# Setup Cloudflare Tunnel
!rm -f cloudflared
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x ./cloudflared
!./cloudflared --version

import subprocess
import time
import re

# Start cloudflared tunnel in the background
port = 8188
command = [f'./cloudflared', 'tunnel', '--url', f'http://localhost:{port}']
cloudflared_process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Wait and extract the public URL from cloudflared's output
public_url = None
timeout_seconds = 20
start_time = time.time()

while time.time() - start_time < timeout_seconds:
    line = cloudflared_process.stdout.readline()
    if not line:
        time.sleep(0.5)
        continue
    
    # Search for the trycloudflare.com URL in the output
    url_match = re.search(r"https?://[a-zA-Z0-9-]+\.trycloudflare\.com", line)
    if url_match:
        public_url = url_match.group(0)
        print(f"✅ Cloudflare tunnel established: {public_url}")
        break

# ---

# Proceed only if the tunnel was successfully created
if not public_url:
    print("❌ Cloudflare tunnel failed to start. Please restart the runtime and try again.")
    cloudflared_process.kill() # Terminate the orphaned process
else:
    # Start ComfyUI with public access
    %cd /ComfyUI
    # The --enable-cors-header flag allows the UI to connect from the cloudflared domain
    !python main.py --listen 0.0.0.0 --port {port} --cuda-device 0 --enable-cors-header "*"